In [27]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, count
import time
import csv

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
#Both Dataframe and RDD implementations are to use 4 spark executors

spark = SparkSession.builder \
    .appName("Query1 DataFrame API") \
    .config("spark.executor.instances", "4") \
    .getOrCreate()

In [23]:
# Query1 Dataframe implementation

# Start timer
start_time = time.time()

# Load Crime Data
crime_data = spark.read.csv("s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2010_to_2019_20241101.csv", header=True, inferSchema=True)

# Filter for "AGGRAVATED ASSAULT"
assault_data = crime_data.filter(col("Crm Cd Desc").contains("AGGRAVATED ASSAULT"))

# Age Groups
categorized = assault_data.withColumn(
    "AgeGroup",
    when(col("Vict Age") < 18, "Children")
    .when((col("Vict Age") >= 18) & (col("Vict Age") <= 24), "Young Adults")
    .when((col("Vict Age") >= 25) & (col("Vict Age") <= 64), "Adults")
    .when(col("Vict Age") > 64, "Seniors")
)

# Group and count
result_df = categorized.groupBy("AgeGroup").agg(count("*").alias("Count")).orderBy(col("Count").desc())

#Show results
result_df.show()

# Stop timer and print elapsed time
elapsed_time = time.time() - start_time
print(f"Execution Time (DataFrame API): {elapsed_time:.2f} seconds")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+-----+
|    AgeGroup|Count|
+------------+-----+
|      Adults|72610|
|Young Adults|23472|
|    Children|10724|
|     Seniors| 3099|
+------------+-----+

Execution Time (DataFrame API): 19.31 seconds

In [30]:
# Show dataset schema to find indexes for RDD
crime_data.printSchema()

# Display sample rows
crime_data.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- DR_NO: integer (nullable = true)
 |-- Date Rptd: string (nullable = true)
 |-- DATE OCC: string (nullable = true)
 |-- TIME OCC: integer (nullable = true)
 |-- AREA : integer (nullable = true)
 |-- AREA NAME: string (nullable = true)
 |-- Rpt Dist No: integer (nullable = true)
 |-- Part 1-2: integer (nullable = true)
 |-- Crm Cd: integer (nullable = true)
 |-- Crm Cd Desc: string (nullable = true)
 |-- Mocodes: string (nullable = true)
 |-- Vict Age: integer (nullable = true)
 |-- Vict Sex: string (nullable = true)
 |-- Vict Descent: string (nullable = true)
 |-- Premis Cd: integer (nullable = true)
 |-- Premis Desc: string (nullable = true)
 |-- Weapon Used Cd: integer (nullable = true)
 |-- Weapon Desc: string (nullable = true)
 |-- Status: string (nullable = true)
 |-- Status Desc: string (nullable = true)
 |-- Crm Cd 1: integer (nullable = true)
 |-- Crm Cd 2: integer (nullable = true)
 |-- Crm Cd 3: integer (nullable = true)
 |-- Crm Cd 4: integer (nullable = true)
 |-- 

In [33]:
#Query 1 RDD implementation

# Start timer
start_time = time.time()

# Load dataset as RDD
crime_rdd = spark.sparkContext.textFile("s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2010_to_2019_20241101.csv")

# Extract header and filter it out
header = crime_rdd.first()
crime_rdd = crime_rdd.filter(lambda row: row != header)

# Parse CSV rows
def parse_csv(line):
    return list(csv.reader([line]))[0]

parsed_rdd = crime_rdd.map(parse_csv)

# Filter for "AGGRAVATED ASSAULT"
assault_rdd = parsed_rdd.filter(lambda row: "AGGRAVATED ASSAULT" in row[9])

# Age groups
age_group_rdd = assault_rdd.map(lambda row: (
    "Children" if int(row[11]) < 18 else
    "Young Adults" if 18 <= int(row[11]) <= 24 else
    "Adults" if 25 <= int(row[11]) <= 64 else
    "Seniors"
))

# Group and count
result_rdd = age_group_rdd.map(lambda group: (group, 1)).reduceByKey(lambda a, b: a + b).sortBy(lambda x: x[1], ascending=False)

# Show results
for group, count in result_rdd.collect():
    print(f"{group}: {count}")

# Stop timer and print elapsed time
elapsed_time = time.time() - start_time
print(f"Execution Time (RDD API): {elapsed_time:.2f} seconds")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Adults: 72610
Young Adults: 23472
Children: 10724
Seniors: 3099
Execution Time (RDD API): 21.01 seconds

In [21]:
print(crime_data.columns)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['DR_NO', 'Date Rptd', 'DATE OCC', 'TIME OCC', 'AREA ', 'AREA NAME', 'Rpt Dist No', 'Part 1-2', 'Crm Cd', 'Crm Cd Desc', 'Mocodes', 'Vict Age', 'Vict Sex', 'Vict Descent', 'Premis Cd', 'Premis Desc', 'Weapon Used Cd', 'Weapon Desc', 'Status', 'Status Desc', 'Crm Cd 1', 'Crm Cd 2', 'Crm Cd 3', 'Crm Cd 4', 'LOCATION', 'Cross Street', 'LAT', 'LON']

In [22]:
print(crime_data.schema)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

StructType([StructField('DR_NO', IntegerType(), True), StructField('Date Rptd', StringType(), True), StructField('DATE OCC', StringType(), True), StructField('TIME OCC', IntegerType(), True), StructField('AREA ', IntegerType(), True), StructField('AREA NAME', StringType(), True), StructField('Rpt Dist No', IntegerType(), True), StructField('Part 1-2', IntegerType(), True), StructField('Crm Cd', IntegerType(), True), StructField('Crm Cd Desc', StringType(), True), StructField('Mocodes', StringType(), True), StructField('Vict Age', IntegerType(), True), StructField('Vict Sex', StringType(), True), StructField('Vict Descent', StringType(), True), StructField('Premis Cd', IntegerType(), True), StructField('Premis Desc', StringType(), True), StructField('Weapon Used Cd', IntegerType(), True), StructField('Weapon Desc', StringType(), True), StructField('Status', StringType(), True), StructField('Status Desc', StringType(), True), StructField('Crm Cd 1', IntegerType(), True), StructField('Crm

### Query2

### i) Data_Frame API

In [25]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, sum, when, row_number, expr
from pyspark.sql.window import Window
import time

# Start timer
start_time = time.time()

# Start Spark session
spark = SparkSession.builder \
    .appName("Query2 DataFrame API") \
    .config("spark.executor.instances", "4") \
    .getOrCreate()

# Load datasets
crime_data_2010_2019 = spark.read.csv(
    "s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2010_to_2019_20241101.csv",
    header=True, inferSchema=True
)
crime_data_2020_present = spark.read.csv(
    "s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2020_to_Present_20241101.csv",
    header=True, inferSchema=True
)

# Combine both datasets
crime_data = crime_data_2010_2019.union(crime_data_2020_present)


# Aggregation logic
aggregated = crime_data.groupBy(
    expr("substring(`DATE OCC`, 7, 4)").alias("YEAR"),  # Extract year from DATE OCC
    col("AREA NAME")
).agg(
    count("*").alias("total_cases"),
    sum(when(~col("Status Desc").isin("UNK", "Invest Cont"), 1).otherwise(0)).alias("closed_cases")  # Non-"UNK"/"Invest Cont" are closed
).withColumn("closed_case_rate", col("closed_cases") / col("total_cases"))

# Define window specification for ranking within each year
window_spec = Window.partitionBy("YEAR").orderBy(col("closed_case_rate").desc())

# Assign rank and filter top 3 precincts per year
ranked = aggregated.withColumn("ranking", row_number().over(window_spec)) \
    .filter(col("ranking") <= 3) \
    .orderBy("YEAR", "ranking")

# Measure the end time for the DataFrame API operations
dataframe_api_end_time = time.time()

# Count rows for showing all results
row_count = ranked.count()

# Show all rows in the output
ranked.show(truncate=False, n=row_count)

# Print DataFrame API execution time
print(f"DataFrame API Execution Time: {dataframe_api_end_time - dataframe_api_start_time:.2f} seconds")



FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+-----------+-----------+------------+-------------------+-------+
|YEAR|AREA NAME  |total_cases|closed_cases|closed_case_rate   |ranking|
+----+-----------+-----------+------------+-------------------+-------+
|2010|Rampart    |8707       |2860        |0.32847134489491214|1      |
|2010|Olympic    |8764       |2762        |0.3151528982199909 |2      |
|2010|Harbor     |9598       |2818        |0.2936028339237341 |3      |
|2011|Olympic    |7988       |2799        |0.35040060090135206|1      |
|2011|Rampart    |8444       |2744        |0.324964471814306  |2      |
|2011|Harbor     |9841       |2806        |0.2851336246316431 |3      |
|2012|Olympic    |8543       |2930        |0.3429708533302119 |1      |
|2012|Rampart    |8626       |2800        |0.3246000463714352 |2      |
|2012|Harbor     |9441       |2786        |0.29509585848956676|3      |
|2013|Olympic    |8305       |2789        |0.3358217940999398 |1      |
|2013|Rampart    |8148       |2616        |0.32106038291605304|2

### ii) SQL API

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import expr
import time

# Start timer
sql_api_start_time = time.time()
# Start Spark session
spark = SparkSession.builder \
    .appName("Query2 SQL API") \
    .config("spark.executor.instances", "4") \
    .getOrCreate()

# Load datasets
crime_data_2010_2019 = spark.read.csv(
    "s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2010_to_2019_20241101.csv",
    header=True, inferSchema=True
)
crime_data_2020_present = spark.read.csv(
    "s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2020_to_Present_20241101.csv",
    header=True, inferSchema=True
)

# Combine datasets
crime_data = crime_data_2010_2019.union(crime_data_2020_present)

# Register the DataFrame as a temporary SQL table
crime_data.createOrReplaceTempView("crime_data")

# SQL query
sql_query = """
    WITH Aggregated AS (
        SELECT
            SUBSTRING(`DATE OCC`, 7, 4) AS YEAR,
            `AREA NAME` AS Precinct,
            COUNT(*) AS total_cases,
            SUM(CASE WHEN `Status Desc` NOT IN ('UNK', 'Invest Cont') THEN 1 ELSE 0 END) AS closed_cases,
            SUM(CASE WHEN `Status Desc` NOT IN ('UNK', 'Invest Cont') THEN 1 ELSE 0 END) / COUNT(*) AS closed_case_rate
        FROM crime_data
        GROUP BY SUBSTRING(`DATE OCC`, 7, 4), `AREA NAME`
    ),
    Ranked AS (
        SELECT
            YEAR,
            Precinct,
            total_cases,
            closed_cases,
            closed_case_rate,
            ROW_NUMBER() OVER (PARTITION BY YEAR ORDER BY closed_case_rate DESC) AS ranking
        FROM Aggregated
    )
    SELECT
        YEAR,
        Precinct,
        total_cases,
        closed_cases,
        closed_case_rate,
        ranking
    FROM Ranked
    WHERE ranking <= 3
    ORDER BY YEAR, ranking
"""



# Execute the SQL query
result = spark.sql(sql_query)

# Count rows to ensure all results are shown
row_count = result.count()

# Show the results
result.show(truncate=False, n=row_count)

# End timer
sql_api_end_time = time.time()

# Print SQL API execution time
print(f"SQL API Execution Time: {sql_api_end_time - sql_api_start_time:.2f} seconds")


csv to parquet transition

In [28]:
from pyspark.sql import SparkSession

# Start Spark session
spark = SparkSession.builder \
    .appName("Save Crime Data as Parquet") \
    .config("spark.executor.instances", "4") \
    .getOrCreate()

# Load datasets
crime_data_2010_2019 = spark.read.csv(
    "s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2010_to_2019_20241101.csv",
    header=True, inferSchema=True
)
crime_data_2020_present = spark.read.csv(
    "s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2020_to_Present_20241101.csv",
    header=True, inferSchema=True
)

# Combine datasets
crime_data = crime_data_2010_2019.union(crime_data_2020_present)

# Save as a single Parquet file to the specified S3 bucket
output_path = "s3://groups-bucket-dblab-905418150721/group28/query2/"
crime_data.repartition(1).write.mode("overwrite").parquet(output_path)

print(f"Data successfully saved to {output_path} in Parquet format.")


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Data successfully saved to s3://groups-bucket-dblab-905418150721/group28/query2/ in Parquet format.

In [30]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import expr
import time

# Start timer
sql_api_start_time = time.time()

# Start Spark session
spark = SparkSession.builder \
    .appName("Query2 SQL API") \
    .config("spark.executor.instances", "4") \
    .getOrCreate()

# Crime Dataset in Parquet format
crime_data = spark.read.parquet(
    "s3://groups-bucket-dblab-905418150721/group28/query2/part-00000-fea3c04b-7961-41ea-8e05-d62534cf766e-c000.snappy.parquet"
)

# Register the DataFrame as a temporary SQL table
crime_data.createOrReplaceTempView("crime_data")

# SQL query
sql_query = """
    WITH Aggregated AS (
        SELECT
            SUBSTRING(`DATE OCC`, 7, 4) AS YEAR,
            `AREA NAME` AS Precinct,
            COUNT(*) AS total_cases,
            SUM(CASE WHEN `Status Desc` NOT IN ('UNK', 'Invest Cont') THEN 1 ELSE 0 END) AS closed_cases,
            SUM(CASE WHEN `Status Desc` NOT IN ('UNK', 'Invest Cont') THEN 1 ELSE 0 END) / COUNT(*) AS closed_case_rate
        FROM crime_data
        GROUP BY SUBSTRING(`DATE OCC`, 7, 4), `AREA NAME`
    ),
    Ranked AS (
        SELECT
            YEAR,
            Precinct,
            total_cases,
            closed_cases,
            closed_case_rate,
            ROW_NUMBER() OVER (PARTITION BY YEAR ORDER BY closed_case_rate DESC) AS ranking
        FROM Aggregated
    )
    SELECT
        YEAR,
        Precinct,
        total_cases,
        closed_cases,
        closed_case_rate,
        ranking
    FROM Ranked
    WHERE ranking <= 3
    ORDER BY YEAR, ranking
"""

# Execute the SQL query
result = spark.sql(sql_query)

# Count rows to ensure all results are shown
row_count = result.count()

# Show the results
result.show(truncate=False, n=row_count)

# End timer
sql_api_end_time = time.time()

# Print SQL API execution time
print(f"SQL API Execution Time: {sql_api_end_time - sql_api_start_time:.2f} seconds")


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+-----------+-----------+------------+-------------------+-------+
|YEAR|Precinct   |total_cases|closed_cases|closed_case_rate   |ranking|
+----+-----------+-----------+------------+-------------------+-------+
|2010|Rampart    |8707       |2860        |0.32847134489491214|1      |
|2010|Olympic    |8764       |2762        |0.3151528982199909 |2      |
|2010|Harbor     |9598       |2818        |0.2936028339237341 |3      |
|2011|Olympic    |7988       |2799        |0.35040060090135206|1      |
|2011|Rampart    |8444       |2744        |0.324964471814306  |2      |
|2011|Harbor     |9841       |2806        |0.2851336246316431 |3      |
|2012|Olympic    |8543       |2930        |0.3429708533302119 |1      |
|2012|Rampart    |8626       |2800        |0.3246000463714352 |2      |
|2012|Harbor     |9441       |2786        |0.29509585848956676|3      |
|2013|Olympic    |8305       |2789        |0.3358217940999398 |1      |
|2013|Rampart    |8148       |2616        |0.32106038291605304|2